# 1. Imports, Cleaning and EDA

## 1.1. Imports

In [1]:
from pySimFinLIB import pySimFin
import pandas as pd
import seaborn as sns
from simfin.names import *
import matplotlib.pyplot as plt
import requests
from dotenv import load_dotenv
import os
import pandas as pd

## 1.2 Data Cleaning

In [2]:
ticker = 'AAPL'
startDate = '2022-01-01'
endDate = '2023-01-01'

In [3]:
psf = pySimFin()

In [4]:
infoDF = psf.getCompanyInfo(ticker)
companyDF = psf.getCompanyList()
pricesDF = psf.getStockPrices(ticker,startDate,endDate)

In [9]:
companyDF

,id,name,ticker,sectorCode,industryName,sectorName,isin
0,1234791,STERLING BANCORP,STL,104002.0,Financial Services,Banks,US85917A1007
3,1253409,Infosys Ltd,INFY,101003.0,Technology,Application Software,US4567881085
4,1291790,ACCELERON PHARMA INC,XLRN,106001.0,Healthcare,Medical Diagnostics & Research,US00434H1086
42,65359,ALEXION PHARMACEUTICALS INC,ALXN,106002.0,Healthcare,Biotechnology,US0153511094
48,76840,"REMEDENT, INC.",REMI,106003.0,Healthcare,Medical Instruments & Equipment,US75954T1043
...,...,...,...,...,...,...,...
6264,18599360,Seanergy Maritime Holdings Corp.,SHIP,100010.0,Industrials,Transportation & Logistics,MHY737601945
6265,18599362,Wix.com Ltd.,WIX,101003.0,Technology,Application Software,IL0011301780
6266,18599364,Sprott Physical Platinum and Palladium Trust,SPPP,104001.0,Financial Services,Asset Management,CA85207Q1046
6267,18599366,Sportradar Group AG,SRAD,101003.0,Technology,Application Software,CH1134239669


In [5]:
stock_data = psf.getStockPrices(ticker, startDate, endDate)
stock_data 

,Common Shares Outstanding,Last Closing Price,Adjusted Closing Price,Highest Price,Lowest Price,Opening Price,Trading Volume
Date,,,,,,,
2022-01-03,16687631000,182.01,178.89,182.88,177.71,177.83,104701220
2022-01-04,16687631000,179.70,176.61,182.94,179.12,182.63,99310438
2022-01-05,16687631000,174.92,171.92,180.17,174.64,179.61,94537602
2022-01-06,16687631000,172.00,169.05,175.30,171.64,172.70,96903955
2022-01-07,16687631000,172.17,169.21,174.14,171.03,172.89,86709147
...,...,...,...,...,...,...,...
2022-12-23,16030382000,131.86,130.35,132.41,129.64,130.92,63814893
2022-12-27,16030382000,130.03,128.54,131.41,128.72,131.38,69007830
2022-12-28,16030382000,126.04,124.60,131.03,125.87,129.67,85438391


In [6]:
def selectMultipleStocks(selected_stocks, startDate, endDate):
    selected_stock_data = {}

    for ticker in selected_stocks:
        # Get stock data for each ticker
        stock_data = psf.getStockPrices(ticker, startDate, endDate)
        stock_data = stock_data['Last Closing Price']
        
        
        # Store stock data in the dictionary with the ticker as the key
        selected_stock_data[ticker] = stock_data

    return pd.DataFrame(selected_stock_data)

In [8]:


selected_stocks = ['AAPL','AMZN','NFLX','GOOG']

psf.selectMultipleStocks(selected_stocks, startDate, endDate)

,AAPL,AMZN,NFLX,GOOG
Date,,,,
2022-01-03,182.01,170.40,597.37,145.07
2022-01-04,179.70,167.52,591.15,144.42
2022-01-05,174.92,164.36,567.52,137.65
2022-01-06,172.00,163.25,553.29,137.55
2022-01-07,172.17,162.55,541.06,137.00
...,...,...,...,...
2022-12-23,131.86,85.25,294.96,89.81
2022-12-27,130.03,83.04,284.17,87.93
2022-12-28,126.04,81.82,276.88,86.46


In [13]:
def tickerFind(nameList,companyDF):
    tickerList = []
    for name in nameList:
        for index, company in companyDF.iterrows():
            if name == company['name']:
                tickerList.append(company['ticker'])
    return tickerList

names = list(companyDF['name'].sample(5))

tickerFind(names,companyDF)

['HARP', 'UDMY', 'AOSL', 'SKWD', 'OPXS']